# Financial Data Analysis – Data Processing 1: Loan Eligibility Prediction

[Link](https://www.kdnuggets.com/2018/09/financial-data-analysis-loan-eligibility-prediction.html)

This notebook is from kdnuggets (link above) about Loan Eligibility Prediction using use Lending club loan data dataset. Lending Club is the world’s largest online marketplace connecting borrowers and investors. An inevitable outcome of lending is default by borrowers. The idea of this tutorial is to create a predictive model that **identifies applicants who are relatively risky for a loan.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
df1 = pd.read_csv('./data/')